In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('project_b_samples_2k.parquet')
df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""shipping_center"",""alternate"":[""fre...",0.996261,"[""http://www.goinpostaljacksonville.com/""]","[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""vehicle_shipping"",""alternate"":[""co...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{""freeform"":""7643 Gate Pkwy Ste 104"",""localit..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Valley Transmission""}","{""primary"":""automotive_repair"",""alternate"":[""a...",0.996261,"[""http://valleytransca.com/""]","[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...","{""primary"":""Valley Transmission""}","{""primary"":""transmission_repair"",""alternate"":[...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{""freeform"":""1158 N 2nd St"",""locality"":""El Ca..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Mazda Nelspruit""}","{""primary"":""car_dealer"",""alternate"":[""automoti...",0.997899,"[""http://bit.ly/NelspruitMazda""]","[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...","{""primary"":""Mazda Nelspruit""}","{""primary"":""used_car_dealer"",""alternate"":[""aut...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{""freeform"":""49 Emnotweni Avenue"",""locality"":..."
3,1407374885933937,844424932146366,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Red Wing - Roswell, GA""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.770000,"[""https://stores.redwingshoes.com/roswell-ga""]",None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...","{""primary"":""Red Wing""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{""freeform"":""875 Mansell Rd Ste B1"",""locality..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Norauto España""}","{""primary"":""automotive_repair"",""alternate"":[""c...",0.996261,"[""https://centro.norauto.es/alacant/finestrat/...","[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Norauto""}","{""primary"":""brake_service_and_repair"",""alterna...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{""freeform"":""Calle Rei Jaume I 9"",""locality"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""lawyer"",""alternate"":[""criminal_def...",0.286713,"[""http://www.petraccimarin.it""]","[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""Business 